我们采用基于Unet的扩散模型并使用EUVP数据集来进行水下图像模型的训练。
EUVP数据集（Enhance Underwater Visual Perception Dataset）是一个专门用于水下图像处理的公开数据集，主要用于研究和开发水下图像增强和恢复技术。

In [ ]:
import torch
import os
from torchvision.utils import save_image
from PIL import Image
from T4_train import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

target_imgs_path = os.path.join(os.getcwd(), 'Attachment', "Attachment 2")
output_imgs_path = os.path.join(os.getcwd(), 'imgs_output')
if not os.path.exists(output_imgs_path):
    os.makedirs(output_imgs_path)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()])
img_size = (256,256)
timesteps = 1000

model = UNet(in_channels=3, out_channels=3).to(device)
model.load_state_dict(torch.load("./diffusion_model.pth"))
model.eval()
diffusion = DiffusionModel(model, img_size, timesteps, device)

for img in os.listdir(target_imgs_path):
    img_path = os.path.join(target_imgs_path, img)
    if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        img = Image.open(img_path).convert('RGB')
        img = transform(img).unsqueeze(0).to(device)
        
        with torch.no_grad():
            generated_img = diffusion.generate(batch_size=1)
        
        output_imgs_path = os.path.join(output_imgs_path, f"generated_{img}")
        save_image(generated_img, output_imgs_path)
        
        print(f'{img} has been generated!')